In [1]:
from muse_maskgit_pytorch import TransformerBlocks, LayerNorm

import torch.nn as nn

import torch

import math

/home/ubuntu/anaconda3/envs/muse/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append("..")

from masked_model import MaskedModel, SequenceModelWrapper

In [3]:
class Transformer(nn.Module):
  def __init__(
    self,
    token_size,
    depth,
    # if None, dim_out = dim_in
    dim_head = 64,
    ff_mult = 4,
  ):
    super().__init__()
    self.token_size = token_size

    self.transformer_blocks = TransformerBlocks(dim = token_size, depth = depth, dim_head = dim_head, ff_mult = ff_mult)
    self.norm = LayerNorm(token_size)

  def forward(self, x, context, context_mask):
    return self.norm(self.transformer_blocks(x, context, context_mask))

In [4]:
def cosine_schedule(t):
    return torch.cos(t * math.pi * 0.5)

In [5]:
model = MaskedModel(
  SequenceModelWrapper(
    Transformer(
      2048,
      24,
    ),
    8192,
    16*16,
    False
  ),
  cosine_schedule,
  0.1,
).cuda()

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


In [10]:
model.sequence_model.sequence_model

Transformer(
  (transformer_blocks): TransformerBlocks(
    (layers): ModuleList(
      (0-23): 24 x ModuleList(
        (0-1): 2 x Attention(
          (norm): LayerNorm()
          (attend): Attend(
            (attn_dropout): Dropout(p=0.0, inplace=False)
          )
          (to_q): Linear(in_features=2048, out_features=512, bias=False)
          (to_kv): Linear(in_features=2048, out_features=1024, bias=False)
          (to_out): Linear(in_features=512, out_features=2048, bias=False)
        )
        (2): Sequential(
          (0): LayerNorm()
          (1): Linear(in_features=2048, out_features=10922, bias=False)
          (2): GEGLU()
          (3): LayerNorm()
          (4): Linear(in_features=5461, out_features=2048, bias=False)
        )
      )
    )
    (norm): LayerNorm()
  )
  (norm): LayerNorm()
)

In [6]:
model(image_ids = torch.rand(1, 10).long().cuda(), text_embeds = torch.rand(1, 5, 768).float().cuda())

tensor(12.0081, device='cuda:0', grad_fn=<AddBackward0>)

In [7]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [8]:
count_parameters(model)

1042585593

In [11]:
x = torch.rand((1, 10, 2))
y = torch.rand((1, 5, 2))

In [14]:
to_q = nn.Linear(2, 4)
to_kv = nn.Linear(2, 8)

In [16]:
q, k, v = to_q(x), *to_kv(y).chunk(2, dim = -1)

In [23]:
emb = nn.Embedding(4, 4)

In [24]:
emb.weight

Parameter containing:
tensor([[ 1.0819, -0.3698, -0.2357, -0.3285],
        [ 0.9892, -1.2152, -0.5616, -0.1422],
        [-0.8046, -0.9390,  1.0748, -1.6716],
        [-0.1133,  0.2292, -0.8661, -2.2202]], requires_grad=True)

In [33]:
emb(torch.Tensor([4]).long())

IndexError: index out of range in self

: 